In [19]:
# Import Libraries
import pandas as pd

jobs = pd.read_csv('./datasets/customer_train.csv')

print(jobs.memory_usage())

Index                        132
student_id                153264
city                      153264
city_development_index    153264
gender                    153264
relevant_experience       153264
enrolled_university       153264
education_level           153264
major_discipline          153264
experience                153264
company_size              153264
company_type              153264
last_new_job              153264
training_hours            153264
job_change                153264
dtype: int64


In [20]:
# Make a copy of the jobs dataframe for transformation
ds_jobs_transformed = jobs.copy()

# Start coding
for col in ds_jobs_transformed.select_dtypes('object').columns:
    print(ds_jobs_transformed[col].value_counts, '\n')

<bound method IndexOpsMixin.value_counts of 0        city_103
1         city_40
2         city_21
3        city_115
4        city_162
           ...   
19153    city_173
19154    city_103
19155    city_103
19156     city_65
19157     city_67
Name: city, Length: 19158, dtype: object> 

<bound method IndexOpsMixin.value_counts of 0        Male
1        Male
2         NaN
3         NaN
4        Male
         ... 
19153    Male
19154    Male
19155    Male
19156    Male
19157     NaN
Name: gender, Length: 19158, dtype: object> 

<bound method IndexOpsMixin.value_counts of 0        Has relevant experience
1         No relevant experience
2         No relevant experience
3         No relevant experience
4        Has relevant experience
                  ...           
19153     No relevant experience
19154    Has relevant experience
19155    Has relevant experience
19156    Has relevant experience
19157     No relevant experience
Name: relevant_experience, Length: 19158, dtype: object> 

<bou

In [21]:
# LONG VERSION

# Transform columns with only two factors to be stored as Booleans
ds_jobs_transformed['job_change'] = ds_jobs_transformed['job_change'].astype('bool')

experience_map = {
    'Has relevant experience': True,
    'No relevant experience': False
}
ds_jobs_transformed['relevant_experience'] = ds_jobs_transformed['relevant_experience'].map(experience_map)

# Transform columns containing integers only to be stored as 32-bit int
ds_jobs_transformed['student_id'] = ds_jobs_transformed['student_id'].astype('int32')
ds_jobs_transformed['training_hours'] = ds_jobs_transformed['training_hours'].astype('int32')

# Transform columns containing floats to be stored as 16-bit floats
ds_jobs_transformed['city_development_index'] = ds_jobs_transformed['city_development_index'].astype('float16')

# Transform nominal categorical data to category dtype
ds_jobs_transformed['gender'] = ds_jobs_transformed['gender'].astype('category')
ds_jobs_transformed['city'] = ds_jobs_transformed['city'].astype('category')
ds_jobs_transformed['major_discipline'] = ds_jobs_transformed['major_discipline'].astype('category')
ds_jobs_transformed['company_type'] = ds_jobs_transformed['company_type'].astype('category')

# Transform ordinal categorical data to category dtype
education_levels = pd.CategoricalDtype(['Primary School','High School','Graduate','Masters','Phd'], ordered=True)
enrollment_map = pd.CategoricalDtype(['no_enrollment','Part time course', 'Full time course'], ordered=True)
experience_cat = pd.CategoricalDtype(['<1'] + list(map(str, range(1, 21))) + ['>20'], ordered=True)
company_size_map = pd.CategoricalDtype(['<10','10-49','50-99','100-499','500-999','1000-4999','5000-9999','10000+'], ordered=True)
lastest_job_map = pd.CategoricalDtype(['never','1','2','3','4','>4'], ordered=True)

ds_jobs_transformed['education_level'] = ds_jobs_transformed['education_level'].astype(education_levels)
ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].astype(enrollment_map)
ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].astype(experience_cat)
ds_jobs_transformed['company_size'] = ds_jobs_transformed['company_size'].astype(company_size_map)
ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].astype(lastest_job_map)

print(ds_jobs_transformed.memory_usage())

# Filter dataframe
ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed['experience'] >= '10') & (ds_jobs_transformed['company_size'] >= '1000-4999')]

Index                       132
student_id                76632
city                      24310
city_development_index    38316
gender                    19290
relevant_experience       19158
enrolled_university       19290
education_level           19370
major_discipline          19378
experience                19890
company_size              19522
company_type              19378
last_new_job              19378
training_hours            76632
job_change                19158
dtype: int64


In [22]:
# EFFICIENT VERSION

# Another copy of the dataframe
ds_jobs = jobs.copy()

# Create a dictionary of columns containing ordered categorical data
ordered_cats = {
    'education_level': ['Primary School','High School','Graduate','Masters','Phd'],
    'enrolled_university': ['no_enrollment','Part time course', 'Full time course'],
    'experience': ['<1'] + list(map(str, range(1, 21))) + ['>20'],
    'company_size': ['<10','10-49','50-99','100-499','500-999','1000-4999','5000-9999','10000+'],
    'last_new_job': ['never','1','2','3','4','>4']
}

# Create a mapping dictionary of columsn containing two-factor categories to convert to Booleans
two_factor_cats = {
    'relevant_experience':{'No relevant experience': False, 'Has relevant experience': True},
    'job_change': {0.0: False, 1.0: True}
}

# Loop through DataFrame columns to efficiently change data types
for coll in ds_jobs:
    if coll in ['relevant_experience','job_change']:
        ds_jobs[coll] = ds_jobs[coll].map(two_factor_cats[coll])
    
    elif coll in ['student_id','training_hours']:
        ds_jobs[coll] = ds_jobs[coll].astype('int32')
    
    elif coll == 'city_development_index':
        ds_jobs[coll] = ds_jobs[coll].astype('float16')
    
    elif coll in ordered_cats.keys():
        category = pd.CategoricalDtype(ordered_cats[coll], ordered=True)
        ds_jobs[coll] = ds_jobs[coll].astype(category)
    
    else:
        ds_jobs[coll] = ds_jobs[coll].astype('category')

In [23]:
# Exploratory

print(ds_jobs['major_discipline'].value_counts().sort_index())

major_discipline
Arts                 253
Business Degree      327
Humanities           669
No Major             223
Other                381
STEM               14492
Name: count, dtype: int64
